In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import plotly.plotly as py
import plotly.graph_objs as go
import time
import pytz
from datetime import datetime
py.sign_in('subhangi', '')
%matplotlib inline

In [10]:
with open('/Users/arpitsabherwal/Desktop/2016-10-01 October 2016 to December 2016.json',encoding='utf-8') as data_file:    
    data_search = json.load(data_file)

In [11]:
with open('/Users/arpitsabherwal/Desktop/2017-01-01 January 2017 to March 2017.json',encoding='utf-8') as data_file:    
    data_search_jan = json.load(data_file)

In [12]:
with open('/Users/arpitsabherwal/Desktop/apple_steps.json',encoding='utf-8') as data_file:    
    data_steps = json.load(data_file)

In [13]:
with open('/Users/arpitsabherwal/Desktop/watch-history.json',encoding='utf-8') as data_file:    
    data_youtube = json.load(data_file)

In [188]:
server_timezone = pytz.timezone('UTC')
new_timezone = pytz.timezone("America/New_York")

In [229]:
def plot_individual_data(x,y):
    y_temp_val=np.argsort(np.array(x))
    x=sorted(x)
    
    y_new_array=[]
    
    for i in range(len(y)):
        y_new_array.append(y[y_temp_val[i]])
    
    data = [go.Scatter(x=x, y=y_new_array,mode = 'lines+markers',name='Search Query')]
    #py.iplot(data) # This plot depicts users steps
    return x,y_new_array,data
    
def plot_step_search(start_time,stop_time):
    x_search_time=np.arange(start_time,stop_time,1000)
    x_search_time=x_search_time.tolist()
    
    x_step_time=np.arange(start_time,stop_time,1000)
    x_step_time=x_step_time.tolist()
    
    
    y_search_true=[]
    for i in range(len(x_search_time)):
        x_search_time[i]=server_timezone.localize(datetime.fromtimestamp(x_search_time[i])).astimezone(new_timezone)
        y_search_true.append(0)
    
    y_step=[]
    for i in range(len(x_step_time)):
        x_step_time[i]=server_timezone.localize(datetime.fromtimestamp(x_step_time[i])).astimezone(new_timezone)
        y_step.append(0)
        
        
    start_time_date = server_timezone.localize(datetime.fromtimestamp(start_time)).astimezone(new_timezone) 
    stop_time_date = server_timezone.localize(datetime.fromtimestamp(stop_time)).astimezone(new_timezone)
    
    for i in range(len(data_search["event"])):
        temp_time=(float(data_search["event"][i]["query"]["id"][0]["timestamp_usec"])/1000000)
        temp_time_date = server_timezone.localize(datetime.fromtimestamp(temp_time)).astimezone(new_timezone) 
        if temp_time_date>start_time_date and temp_time_date<stop_time_date:
            x_search_time.append(temp_time_date)
            y_search_true.append(500)

    for i in range(len(data_search_jan["event"])):
        temp_time=(float(data_search_jan["event"][i]["query"]["id"][0]["timestamp_usec"])/1000000)
        temp_time_date = server_timezone.localize(datetime.fromtimestamp(temp_time)).astimezone(new_timezone) 
        if temp_time_date>start_time_date and temp_time_date<stop_time_date:
            x_search_time.append(temp_time_date)
            y_search_true.append(500)
    
    x_search,new_y_val_search,search_data=plot_individual_data(x_search_time,y_search_true)
    
    for i in range(len(data_steps["HealthData"]["Record"])):
        if data_steps["HealthData"]["Record"][i]["-type"]=="HKQuantityTypeIdentifierStepCount":
            temp_time=time.mktime(datetime.strptime(data_steps["HealthData"]["Record"][i]["-startDate"][:-6], "%Y-%m-%d %H:%M:%S").timetuple())
            temp_time_date = server_timezone.localize(datetime.fromtimestamp(temp_time)).astimezone(new_timezone) 
            if temp_time_date>start_time_date and temp_time_date<stop_time_date:
                x_step_time.append(temp_time_date)
                y_step.append(int(data_steps["HealthData"]["Record"][i]["-value"]))
                #y_step.append(1)
    
    x_step,new_y_val_step,step_data=plot_individual_data(x_step_time,y_step)
    
    combined_data=[go.Scatter(x=x_step, y=new_y_val_step,mode = 'lines+markers',name='Number of Steps'),go.Scatter(x=x_search, y=new_y_val_search,mode = 'lines+markers',name='Search Query')]
    return search_data,step_data,combined_data
    
    
    

In [255]:
search_data,step_data,combined_data=plot_step_search(1489507200,1489593600) # To plot the graph for different dates

In [256]:
py.iplot(combined_data)

In [112]:
step_time={}

In [113]:
for i in range(len(data_steps["HealthData"]["Record"])):
    if data_steps["HealthData"]["Record"][i]["-type"]=="HKQuantityTypeIdentifierStepCount":
        temp_time=time.mktime(datetime.strptime(data_steps["HealthData"]["Record"][i]["-endDate"][:-6], "%Y-%m-%d %H:%M:%S").timetuple())
        temp_time_date = server_timezone.localize(datetime.fromtimestamp(temp_time)).astimezone(new_timezone) 
        temp_time=datetime.timestamp(temp_time_date)
        step_time[temp_time]=int(data_steps["HealthData"]["Record"][i]["-value"])

In [114]:
import collections
step_time = collections.OrderedDict(sorted(step_time.items()))

In [115]:
step_time.pop(1475184463.0)
step_time.pop(1475184848.0)

7

In [116]:
date=1478232000 # 4 November 2016

In [117]:
step_date={}

In [118]:
date=1478232000 # 4 November 2016
step_count=0
for key, value in step_time.items():
    if float(key)>=float(date) and float(key)<float(date+86400):
        step_count=step_count+value
    else:
        step_date[date]=step_count
        date=date+86400
        step_count=value

In [119]:
step_date = collections.OrderedDict(sorted(step_date.items()))

In [120]:
x_step_date=[]
y_no_steps=[]
for key, value in step_date.items():
    x_step_date.append(datetime.fromtimestamp(key))
    y_no_steps.append(value)

In [121]:
data_step_date = [go.Scatter(x=x_step_date, y=y_no_steps,mode = 'lines+markers',name='Steps in a day')]

In [122]:
py.iplot(data_step_date) # step count for each dayvarious months

In [128]:
search_time={}

In [129]:
for i in range(len(data_search["event"])):
    temp_time=int(float(data_search["event"][i]["query"]["id"][0]["timestamp_usec"])/1000000)
    temp_time_date = server_timezone.localize(datetime.fromtimestamp(temp_time)).astimezone(new_timezone) 
    temp_time=datetime.timestamp(temp_time_date)
    search_time[temp_time]=500

In [130]:
for i in range(len(data_search_jan["event"])):
    temp_time=int(float(data_search_jan["event"][i]["query"]["id"][0]["timestamp_usec"])/1000000)
    temp_time_date = server_timezone.localize(datetime.fromtimestamp(temp_time)).astimezone(new_timezone) 
    temp_time=datetime.timestamp(temp_time_date)
    search_time[temp_time]=500

In [131]:
import collections
search_time = collections.OrderedDict(sorted(search_time.items()))

In [132]:
search_start_date=1475294400 # 1st Oct 2016

In [133]:
search_date={}

In [134]:
search_count=0
for key, value in search_time.items():
    if float(key)>=float(search_start_date) and float(key)<float(search_start_date+86400):
        search_count=search_count+1
    else:
        search_date[search_start_date]=search_count
        search_start_date=search_start_date+86400
        search_count=0
        

In [135]:
search_date = collections.OrderedDict(sorted(search_date.items()))

In [136]:
x_search_date=[]
y_no_search=[]
for key, value in search_date.items():
    x_search_date.append(datetime.fromtimestamp(key))
    y_no_search.append(value)

In [137]:
data_search_date = [go.Scatter(x=x_search_date, y=y_no_search,mode = 'lines+markers',name='Searches in a day')]

In [138]:
py.iplot(data_search_date) # search count for each dayvarious months

In [199]:
step_search_each_date={}

In [200]:
for key, value in step_date.items():
    try:
        step_search_each_date[key]=[step_date[key],search_date[key]]
    except KeyError:
        break

In [201]:
step_search_each_date = collections.OrderedDict(sorted(step_search_each_date.items()))

In [202]:
import csv

with open('data_step_search.csv', 'w') as csvfile:
    fieldnames = ['date', 'step','search']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for key, value in step_search_each_date.items():    
        writer.writerow({'date': str(datetime.fromtimestamp(key)), 'step': step_date[key],'search':search_date[key]})